# NTLK BOOK. Chapter 3.

In [ ]:
%pprint

In [ ]:
import nltk

import re
from bs4 import BeautifulSoup
from urllib import request

### Exercise 2.

We can use the slice notation to remove morphological endings on words. For example, 'dogs'[:-1] removes the last character of dogs, leaving dog. Use slice notation to remove the affixes from these words (we've inserted a hyphen to indicate the affix boundary, but omit this from your strings): dish-es, run-ning, nation-ality, un-do, pre-heat.

In [ ]:
def remove_affix(word, suffix=True, prefix=False):
    if suffix:
        hyphen = word.rfind('-')
        word = word[:hyphen]
    if prefix:
        hyphen = word.find('-')
        word = word[hyphen+1:]
    return word

print(remove_affix('dish-es'))
print(remove_affix('run-ning'))
print(remove_affix('nation-ality'))
print(remove_affix('un-do', False, True))
print(remove_affix('pre-heat', False, True))
print(remove_affix('un-do-ing', prefix=True))
print(remove_affix('cat', suffix=False))

### Exercise 3.

We saw how we can generate an IndexError by indexing beyond the end of a string. Is it possible to construct an index that goes too far to the left, before the start of the string?

In [ ]:
word = "NLTK"

try:
    word[-6] # 6th character from the right doesn't exist, so an IndexError is returned.
except IndexError:
    print("Looks like there's an index error here.")    

### Exercise 5.

In [ ]:
monty = 'Monty Python'
monty[::-1]

Como los dos primeros parámetros no están indicados, se reproduce la cadena entera. El tercer parámetro indica cada cuántos caracteres queremos reproducir. Con `1` indicamos que nos interesan todos los caracteres, y como el valor es negativo, la cadena se va a leer desde el final.  

### Exercise 6.

Describe the class of strings matched by the following regular expressions.

    [a-zA-Z]+                      uno o más caracteres alpha
    [A-Z][a-z]*                    una mayúscula seguida de zero, una o más minúsculas
    p[aeiou]{,2}t                  una 'p' seguida de zero, una o dos vocales seguidas de una 't'
    \d+(\.\d+)?                    uno o más dígitos seguidos o no de (un punto seguido de uno o más dígitos) 
    ([^aeiou][aeiou][^aeiou])*     (nada) o (un caracter que no sea una vocal seguido de una vocal seguida de un
                                   caracter que no sea una vocal)
    \w+|[^\w\s]+                   (uno o más cracteres alfanuméricos) o (uno o más caracteres que no sean  ni
                                   alfanuméricos ni de tipo whitespace)

### Exercise 7.

Write regular expressions to match the following classes of strings:

        1. A single determiner (assume that a, an, and the are the only determiners).

In [ ]:
regexp = r'\b([aA][n]?|[tT]he)\b'
re.findall(regexp, 'the tank Anna scandal then an a April The')

       2. An arithmetic expression using integers, addition, and multiplication, such as 2*3+8.

In [ ]:
regexp = r'(?:^|\s)(\d+(?:\*\d+\+|\+\d+\*)\d+)(?:\s|[\.,!\?]+ |$)'
#  matches expressions that
    # contain one '+' and one '*', in any order
    # don't contain any '-'
    # open the string or are preceded by a whitespace
    # close the string or are followed by a whitespace or punctuation mark, followed by a space char.
seq = '9+23*45 10*23+45?! 666*2+14,p 9*237+11+*3+2 1+2+4  600*93+5'

def find_with_overlap(regexp, seq, capgroup_order=1):
    """ Returns a list of overlapping matches of a regular expression
    found in a string.
    If the regular expression has several capturable groups, only the match 
    for one of the groups (the first one, by default) is considered. 
    """
    results=[]
    while True:
        match = re.search(regexp, seq)
        if match is None:
            break
        results.append(match.groups()[capgroup_order-1])
        seq = seq[match.end():]
    return results

print(find_with_overlap(regexp, seq))

### Exercise 13.

What is the difference between calling split on a string with no argument or with ' ' as the argument, e.g. sent.split() versus sent.split(' ')? What happens when the string being split contains tab characters, consecutive space characters, or a sequence of tabs and spaces? (In IDLE you will need to use '\t' to enter a tab character.)

In [ ]:
sent = "Back in the 90s,\tI was in a very famous\n\n TV show."

print("Splitting with no argument:")
print(sent.split())
print("Splitting with one space argument:")
print(sent.split(' '))

`.split()` sin argumentos siempre separa la cadena en los espacios, las tabulaciones y los saltos de línea. Cualquier secuencia de éstos, siempre dará lugar a una única separación.
Si le pasamos un argumento, `.split()` utilizará un algoritmo diferente del que acabamos de describir: separará en cada una de las ocurencias de este argumento. Por ejemplo, si el argumento es un espacio, hará un separación cada vez que encuentre un espacio, y no considerará las tabulaciones y saltos de línea como espacios.

### Exercise 14.

Create a variable words containing a list of words. Experiment with words.sort() and sorted(words). What is the difference?

In [ ]:
my_list = 'Ziggy played guitar, jamming good with Weird and Gilly and the spiders from Mars.'.split()
sorted(my_list)

In [ ]:
my_list

In [ ]:
my_list.sort()

In [ ]:
my_list

`sorted(my_list)` nos devuelve una nueva lista ordenada, mientras que la lista original sigue igual.  

`my_list.sort()` ordena cambia la lista original, ordenándolo, pero no la devuelve.

### Exercise 17.

What happens when the formatting strings %6s and %-6s are used to display strings that are longer than six characters?

In [ ]:
print("{:>6}".format("py"))
print("{:6}{}".format("py", "thon"))

Si usamos el nuevo método de formatear las cadenas, tenemos que sustituir %6s y %-6s por :>6 y :6 respectivamente.   

Lo que hacen es imprimir x espacios antes o después de la cadena, respectivamente, donde x corresponde a `6 menos la longitud de la cadena`. En nuestro caso, la longitud de `py` es 2, por lo que se imprimirán 4 espacios.

### Exercise 18.

Read in some text from a corpus, tokenize it, and print the list of all wh-word types that occur. (wh-words in English are used in questions, relative clauses and exclamations: who, which, what, and so on.) Print them in order. Are any words duplicated in this list, because of the presence of case distinctions or punctuation?

In [ ]:
wh_words = ("what", "where", "when", "why", "who", "which", "whose", "whom")

def normalize_token(token):
    """removes non-alpha cars and lowercases the token."""
    return re.compile('[^a-zA-Z]').sub('', token).lower()

url = "http://www.gutenberg.org/cache/epub/7028/pg7028.txt"
response = request.urlopen(url)
raw = response.read().decode('utf8')

first = raw.find("THE CLICKING OF CUTHBERT")
last = raw.rfind("End of Project Gutenberg's The Clicking of Cuthbert")
raw = raw[first:last]

tokens = nltk.word_tokenize(raw)

wh_tokens = [token for token in tokens 
            if normalize_token(token) in wh_words]


wh_tokens = sorted(set(wh_tokens), key=lambda s: normalize_token(s))
wh_tokens

### Exercise 19.

 Create a file consisting of words and (made up) frequencies, where each line consists of a word, the space character, and a positive integer, e.g. fuzzy 53. Read the file into a Python list using open(filename).readlines(). Next, break each line into its two fields using split(), and convert the number into an integer using int(). The result should be a list of the form: [['fuzzy', 53], ...].

In [ ]:
with open('../data/2_19.txt') as f:
    raw = [line.strip() for line in f.readlines()]
    
freqs = [[word.split()[0], int(word.split()[1])] for word in raw]
    
freqs

### Exercise 20.

Write code to access a favorite webpage and extract some text from it. For example, access a weather site and extract the forecast top temperature for your town or city today.

In [ ]:
url = "http://www.aemet.es/es/eltiempo/prediccion/municipios/madrid-id28079"
html = request.urlopen(url).read()
soup = BeautifulSoup(html, 'lxml')
divs = soup.findAll("div", { 'class' : 'no_wrap'})

for div in divs:
    if div.get_text()[-2:] == '°C': # 1st "no_wrap" div with content ending in '°C' corresponds to the current temp.
        print (div.get_text())
        break